## Reinforcement Learning

Im Gegensatz zum konventionellen Machine Learning gibt es beim Reinforcement Learning keine vorgegebenen Daten. Die Daten für das Trainieren des Models werden während des Prozesses generiert.

Begriffe:

- Umgebung
    - Erzeugt kontinuierlich Trainingsdaten und gibt diese an das Model zurück
    - Effektiv das Programm, welches trainiert werden soll
 
- Prozess
    - Schleife die das Programm ausführt
    - Nimmt die Daten der Umgebung und gibt diese an den Agenten weiter

- Agent
    - Kennt die Regeln des Lernens (wie bekommen wir die Belohnung?)
    - Führt die Actions aus (Beeinflussen der Umgebung)
    - Die Umgebung gibt Punkte als Belohnung, je nach dem ob die Action positiv oder negativ war
 
Packages:

- Gym(nasium)
    - Stellt Umgebung und Spaces bereit

- Stable Baselines 3
    - Stellt viele Extras zur Verfügung die beim Reinforcement hilfreich sind

In [2]:
import gymnasium as gym

## Vordefinierte Umgebungen

Von OpenAI gibt es einige vordefinierte Umgebungen

Eine davon ist die "CartPole" Umgebung

Bei dieser geht es darum, eine Stange die auf einem Kart montiert ist möglichst gerade zu halten

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [12]:
env = gym.make("CartPole-v1")

## Inhalte der Umgebung

Zwei Oberbegriffe: Spaces, Funktionen

Spaces:
- Observation Space: Die Daten die beobachtet werden (der State nach einem Step)
- Action Space: Die möglichen Aktionen des Agenten (z.B. Links, Rechts)
- Box, Discrete, Tuple, Dict, ...

Funktionen:
- step(): Macht einen Schritt, berechnet die Belohnung anhand des States
- reset(): Setzt die Umgebung für einen neuen Versuch
- render(): Zeichnet die UI (falls vorhanden)
- close(): Wird am Ende der Umgebung aufgerufen, wenn noch Dinge abgebaut werden müssen (z.B. UI)

In [6]:
durchgaenge = 5
for x in range(durchgaenge):
    state = env.reset()  # Gibt den Startstate zurück
    score = 0  # Score sollte 200 erreichen

    while True:
        # env.render()
        action = env.action_space.sample()  # Random Action auswählen
        new_state, reward, done, term, info = env.step(action)  # Action an die Umgebung weitergeben
        score += reward
        if done or term:
            break

    print(f"Durchgang {x}: Score: {score}")

Durchgang 0: Score: 11.0
Durchgang 1: Score: 15.0
Durchgang 2: Score: 11.0
Durchgang 3: Score: 23.0
Durchgang 4: Score: 14.0


In [69]:
env.action_space

Discrete(2)

In [68]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

## RL Modell Trainieren

Für ein RL Modell brauchen wir wieder ein paar Dinge:

- Trainingsalgorithmus
    - PPO: Proximal Policy Optimization
    - https://stable-baselines.readthedocs.io/en/master/guide/algos.html
 
- Policy
    - Bestimmt wie der Algorithmus lernt
    - https://stable-baselines.readthedocs.io/en/master/modules/policies.html
 
- (Optional) DummyVecEnv
    - Simulation mehrere Umgebungen und Agenten
    - Nützlich um Zeit zu sparen, kostet mehr Resourcen

In [7]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [31]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)  # Mlp: Multilayer Perceptron -> 2x 64 Dense Nodes

Using cpu device


In [32]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 849  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 527         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008665776 |
|    clip_fraction        | 0.0818      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00215    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.64        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0135     |
|    value_loss           | 53.8        |
-----------------------------------------
----------------------------------

In [37]:
model.save("CartPoleModel.zip")

## Modell testen

In [40]:
env = gym.make("CartPole-v1")

In [41]:
model = PPO.load("CartPoleModel.zip", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [43]:
durchgaenge = 20
for x in range(durchgaenge):
    state = env.reset()  # Gibt den Startstate zurück
    score = 0  # Score sollte 200 erreichen

    while True:
        # env.render()
        if len(state) == 2:
            action = model.predict(state[0])  # Random Action auswählen
        else:
            action = model.predict(state)
        state, reward, done, term, info = env.step(action[0])  # Action an die Umgebung weitergeben
        score += reward
        if done or term:
            break

    print(f"Durchgang {x}: Score: {score}")

Durchgang 0: Score: 500.0
Durchgang 1: Score: 500.0
Durchgang 2: Score: 500.0
Durchgang 3: Score: 500.0
Durchgang 4: Score: 500.0
Durchgang 5: Score: 500.0
Durchgang 6: Score: 500.0
Durchgang 7: Score: 500.0
Durchgang 8: Score: 500.0
Durchgang 9: Score: 500.0
Durchgang 10: Score: 500.0
Durchgang 11: Score: 500.0
Durchgang 12: Score: 500.0
Durchgang 13: Score: 500.0
Durchgang 14: Score: 500.0
Durchgang 15: Score: 500.0
Durchgang 16: Score: 500.0
Durchgang 17: Score: 500.0
Durchgang 18: Score: 500.0
Durchgang 19: Score: 500.0
